In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[8]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.00,0.128929,1.248,0.0,0.0
1,5226.83,0.611315,1.192,0.0,0.0
2,4987.34,0.589212,1.129,0.0,0.0
3,4879.48,0.579258,1.040,0.0,0.0
4,4909.53,0.582031,0.850,0.0,0.0
...,...,...,...,...,...
43819,7530.86,0.823954,4.503,0.0,0.0
43820,6585.61,0.736717,4.342,0.0,0.0
43821,6108.25,0.692661,4.219,0.0,0.0
43822,5657.63,0.651074,4.143,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[8]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_GR,Temp_GR
0,0.00,1.248
1,5226.83,1.192
2,4987.34,1.129
3,4879.48,1.040
4,4909.53,0.850
...,...,...
43819,7530.86,4.503
43820,6585.61,4.342
43821,6108.25,4.219
43822,5657.63,4.143


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_GR,Temp_GR,Lag-2,Lag-7
0,5869.19,-0.927,5577.53,0.00
1,5707.95,-0.990,5342.29,5226.83
2,5552.57,-1.046,5158.07,4987.34
3,5562.97,-1.198,5151.00,4879.48
4,6053.18,-1.348,5253.96,4909.53
...,...,...,...,...
43651,7530.86,4.503,6812.94,6303.19
43652,6585.61,4.342,6366.50,5662.10
43653,6108.25,4.219,5939.61,5237.44
43654,5657.63,4.143,5327.10,4764.68


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 28s 18ms/step - loss: 0.0112 - mape: 16333.5156 - mae: 0.0809 - val_loss: 0.0058 - val_mape: 9.5166 - val_mae: 0.0609
Epoch 2/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0073 - mape: 16059.9873 - mae: 0.0674 - val_loss: 0.0031 - val_mape: 7.5206 - val_mae: 0.0459
Epoch 3/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0042 - mape: 15363.1035 - mae: 0.0495 - val_loss: 0.0086 - val_mape: 11.3239 - val_mae: 0.0736
Epoch 4/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0033 - mape: 15025.0957 - mae: 0.0427 - val_loss: 0.0069 - val_mape: 10.3606 - val_mae: 0.0678
Epoch 5/100
1105/1105 [==============================] - 19s 18ms/step - loss: 0.0028 - mape: 15112.0732 - mae: 0.0390 - val_loss: 0.0086 - val_mape: 11.3095 - val_mae: 0.0743
Epoch 6/100
1105/1105 [==============================] - 19s 18ms/step - loss: 0.0026 - mape: 14767.5566 - mae: 0.0370 - v

1105/1105 [==============================] - 18s 16ms/step - loss: 0.0013 - mape: 13593.2900 - mae: 0.0239 - val_loss: 0.0063 - val_mape: 9.7029 - val_mae: 0.0622
Epoch 48/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0013 - mape: 13125.2295 - mae: 0.0244 - val_loss: 0.0057 - val_mape: 9.3885 - val_mae: 0.0598
Epoch 49/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0012 - mape: 13195.4180 - mae: 0.0238 - val_loss: 0.0059 - val_mape: 9.1361 - val_mae: 0.0591
Epoch 50/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0012 - mape: 13546.5439 - mae: 0.0236 - val_loss: 0.0057 - val_mape: 8.8270 - val_mae: 0.0574
Epoch 51/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0012 - mape: 13430.7148 - mae: 0.0237 - val_loss: 0.0061 - val_mape: 9.4194 - val_mae: 0.0609
Epoch 52/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0013 - mape: 13195.1816 - mae: 0.0240 - val_loss: 0

1105/1105 [==============================] - 18s 16ms/step - loss: 0.0010 - mape: 11276.5283 - mae: 0.0219 - val_loss: 0.0055 - val_mape: 9.2855 - val_mae: 0.0595
Epoch 94/100
1105/1105 [==============================] - 17s 16ms/step - loss: 9.8774e-04 - mape: 9553.0244 - mae: 0.0216 - val_loss: 0.0059 - val_mape: 9.6043 - val_mae: 0.0616
Epoch 95/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.6289e-04 - mape: 10155.1689 - mae: 0.0214 - val_loss: 0.0053 - val_mape: 8.8200 - val_mae: 0.0567
Epoch 96/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0010 - mape: 10791.8682 - mae: 0.0218 - val_loss: 0.0055 - val_mape: 8.8635 - val_mae: 0.0570
Epoch 97/100
1105/1105 [==============================] - 17s 16ms/step - loss: 9.3952e-04 - mape: 9937.3428 - mae: 0.0212 - val_loss: 0.0057 - val_mape: 9.3260 - val_mae: 0.0595
Epoch 98/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0010 - mape: 11405.0586 - mae: 0.0218 - v

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1864.2898907661438


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 13ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.50400984, 0.4912454 , 0.4919544 , ..., 0.6016162 , 0.5554011 ,
       0.53100777], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.63924662, 0.63049565, 0.62862401, ..., 0.69266145, 0.65107361,
       0.62151766])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,5529.48,4064.138184
1,5434.66,3925.830322
2,5414.38,3933.512695
3,5494.99,4070.795410
4,5933.21,4335.573730
...,...,...
4363,7530.86,6061.744629
4364,6585.61,5597.503418
4365,6108.25,5121.740234
4366,5657.63,4620.981934


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

9.874274616774242